# Pré processamento do dataset de tarefas

In [1]:
import pandas as pd
import numpy as np

In [2]:
try:
    df = pd.read_csv(
        '../data/tasks.csv',
        encoding='utf-8',
        sep=",",
        quotechar='"',
        on_bad_lines='skip',      # ignora linhas corrompidas
        dtype=str,               # carrega tudo como string, depois ajusta
        engine='c'          # usa engine python que é mais tolerante
    )
    print(f"CSV carregado com engine C! {len(df)} linhas encontradas.")

except Exception as e:
    print(f"Erro ao ler CSV: {e}")

CSV carregado com engine C! 7232 linhas encontradas.


In [3]:
# Verificar informações básicas do dataset
if df is not None:
    print("=== INFORMAÇÕES BÁSICAS DO DATASET ===")
    print(f"Shape: {df.shape}")
    print(f"\nTipos de dados:")
    print(df.dtypes)
    print(f"\nPrimeiras 5 linhas:")
    print(df.head())
    print(f"\nValores nulos por coluna:")
    print(df.isnull().sum())
else:
    print("DataFrame não foi carregado corretamente.")

=== INFORMAÇÕES BÁSICAS DO DATASET ===
Shape: (7232, 17)

Tipos de dados:
SUMMARY                          object
DTSTART                          object
DTEND                            object
DUE                              object
NOTES                            object
ATTENDEE                         object
LOCATION                         object
PRIORITY                         object
URL                              object
CALENDAR                         object
UID                              object
ORGANIZER                        object
CATEGORIES                       object
DURATION                         object
REPLACES RECURRENT EVENT FROM    object
CANCELLED                        object
CREATED                          object
dtype: object

Primeiras 5 linhas:
  SUMMARY           DTSTART             DTEND  DUE NOTES ATTENDEE LOCATION  \
0       🍳  07/01/2025 08:00  07/01/2025 08:20  NaN   NaN      NaN      NaN   
1       🧋  07/01/2025 10:00  07/01/2025 10:05  NaN   Na

In [4]:
# Analisar as colunas de data especificamente
if df is not None:
    print("=== ANÁLISE DAS COLUNAS DE DATA ===")

    # Verificar formatos de data
    date_columns = ['DTSTART', 'DTEND', 'DUE', 'CREATED']

    for col in date_columns:
        if col in df.columns:
            print(f"\n{col}:")
            print(f"  - Valores únicos (sample): {df[col].dropna().unique()[:5]}")
            print(f"  - Valores nulos: {df[col].isnull().sum()}")
            print(f"  - Total não-nulos: {df[col].notna().sum()}")

    # Verificar colunas categóricas importantes
    print(f"\n=== COLUNAS CATEGÓRICAS ===")
    categorical_cols = ['CALENDAR', 'CATEGORIES', 'PRIORITY']

    for col in categorical_cols:
        if col in df.columns:
            print(f"\n{col}:")
            values = df[col].value_counts().head(10)
            print(values)

=== ANÁLISE DAS COLUNAS DE DATA ===

DTSTART:
  - Valores únicos (sample): ['07/01/2025 08:00' '07/01/2025 10:00' '07/01/2025 12:30'
 '07/01/2025 16:00' '07/01/2025 18:00']
  - Valores nulos: 0
  - Total não-nulos: 7232

DTEND:
  - Valores únicos (sample): ['07/01/2025 08:20' '07/01/2025 10:05' '07/01/2025 13:00'
 '07/01/2025 16:15' '07/01/2025 18:05']
  - Valores nulos: 2
  - Total não-nulos: 7230

DUE:
  - Valores únicos (sample): []
  - Valores nulos: 7232
  - Total não-nulos: 0

CREATED:
  - Valores únicos (sample): ['07/01/2025 10:12' '07/01/2025 10:14' '07/01/2025 10:15'
 '07/01/2025 10:16' '07/01/2025 10:17']
  - Valores nulos: 0
  - Total não-nulos: 7232

=== COLUNAS CATEGÓRICAS ===

CALENDAR:
CALENDAR
Alimentação    2323
Trabalho       1255
PUC            1219
Exercícios      870
Pessoal         564
PUC 👻           514
Saúde           171
Exames          117
Social           52
Compromisso      49
Name: count, dtype: int64

CATEGORIES:
Series([], Name: count, dtype: int64)

PR

In [5]:
# Converter colunas de data para datetime
if df is not None:
    print("=== CONVERTENDO COLUNAS DE DATA ===")

    # Função para converter data de forma segura
    def safe_datetime_convert(series, format_str='%d/%m/%Y %H:%M'):
        try:
            return pd.to_datetime(series, format=format_str, errors='coerce')
        except:
            return pd.to_datetime(series, errors='coerce')

    # Converter colunas de data
    date_columns = ['DTSTART', 'DTEND', 'CREATED']

    for col in date_columns:
        if col in df.columns:
            print(f"Convertendo {col}...")
            df[col] = safe_datetime_convert(df[col])
            print(f"  - Convertidos: {df[col].notna().sum()}")
            print(f"  - Erros/Nulos: {df[col].isna().sum()}")

    # Converter DURATION para numérico
    if 'DURATION' in df.columns:
        print("Convertendo DURATION...")
        df['DURATION'] = pd.to_numeric(df['DURATION'], errors='coerce')
        print(f"  - Convertidos: {df['DURATION'].notna().sum()}")
        print(f"  - Estatísticas: min={df['DURATION'].min():.2f}, max={df['DURATION'].max():.2f}, média={df['DURATION'].mean():.2f}")

    print("\n✅ Conversões concluídas!")

=== CONVERTENDO COLUNAS DE DATA ===
Convertendo DTSTART...
  - Convertidos: 7232
  - Erros/Nulos: 0
Convertendo DTEND...
  - Convertidos: 7230
  - Erros/Nulos: 2
Convertendo CREATED...
  - Convertidos: 7232
  - Erros/Nulos: 0
Convertendo DURATION...
  - Convertidos: 7230
  - Estatísticas: min=0.08, max=435.00, média=3.20

✅ Conversões concluídas!


## COLUNA: SUMMARY

### Análise

In [6]:
print("=== ANÁlISE DA COLUNA SUMMARY ===")

print(f"Quantidade de valores únicos: {df['SUMMARY'].nunique()}")
# print(f"Valores únicos: {df['SUMMARY'].unique()}")

# Exportar valores únicos da coluna SUMMARY para uma tabela CSV e usar uma LLM para clusterizar informacoes parecidas
unique_summary = df[['SUMMARY', 'CALENDAR']].drop_duplicates().rename(columns={'SUMMARY': 'TITLE'})
unique_summary = unique_summary.sort_values(by='TITLE').reset_index(drop=True)
unique_summary.to_csv('../data/title_calendar_unique.csv', index=False, encoding='utf-8')
print("Valores únicos exportados para ../data/title_unique.csv")


=== ANÁlISE DA COLUNA SUMMARY ===
Quantidade de valores únicos: 376
Valores únicos exportados para ../data/title_unique.csv


### Tratando valores da coluna SUMMARY que possuem dois valores na coluna CALENDAR

In [7]:
# Coluna SUMMARY possui 30 valores que tem duas variações em 'CALENDAR'
print("=== TRATANDO Duplicatas da coluna SUMMARY ===")
df['CALENDAR'] = df['CALENDAR'].replace('PUC 👻', 'PUC')

# Corrigindo os valores duplicados de 'Academia'
df.loc[(df['SUMMARY'] == 'Academia') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'Exercícios' # 2
print(f'Valores de CALENDAR para "Academia": {df.loc[df['SUMMARY'] == 'Academia', 'CALENDAR'].unique()}')

# Corrigindo os valores duplicados de 'Aldeia'
df.loc[(df['SUMMARY'] == 'Aldeia') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'Social' # 2
print(f'Valores de CALENDAR para "Aldeia": {df.loc[df['SUMMARY'] == 'Aldeia', 'CALENDAR'].unique()}')

# Corrigindo os valores duplicados de 'Aula'
df.loc[(df['SUMMARY'] == 'Aula') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'PUC' # 2
print(f'Valores de CALENDAR para "Aula": {df.loc[df['SUMMARY'] == 'Aula', 'CALENDAR'].unique()}')

# Corrigindo os valores duplicados de 'BD: T2 G2'
df.loc[(df['SUMMARY'] == 'BD: T2 G2') & (df['CALENDAR'] == 'PUC'), 'CALENDAR'] = 'Exames' # 2
print(f'Valores de CALENDAR para "BD: T2 G2": {df.loc[df['SUMMARY'] == 'BD: T2 G2', 'CALENDAR'].unique()}')

# Corrigindo os valores duplicados de 'BIRUSAMBA DE VERÃO'
df.loc[(df['SUMMARY'] == 'BIRUSAMBA DE VERÃO'), 'CALENDAR'] = 'Social' # 2
print(f'Valores de CALENDAR para "BIRUSAMBA DE VERÃO": {df.loc[df['SUMMARY'] == 'BIRUSAMBA DE VERÃO', 'CALENDAR'].unique()}')

# Corrigindo os valores duplicados de De-Para
df.loc[(df['SUMMARY'] == 'De-Para') & (df['CALENDAR'] == 'Exames'), 'CALENDAR'] = 'PUC' # 2
print(f'Valores de CALENDAR para "De-Para": {df.loc[df["SUMMARY"] == "De-Para", "CALENDAR"].unique()}')

# Corrigindo os valores duplicados de 'Muay Thai'
df.loc[(df['SUMMARY'] == 'Muay Thai') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'Exercícios' # 2
print(f'Valores de CALENDAR para "Muay Thai": {df.loc[df["SUMMARY"] == "Muay Thai", "CALENDAR"].unique()}')

# Corrigindo valores duplicados de 'Psicóloga'
df.loc[(df['SUMMARY'] == 'Psicóloga') & (df['CALENDAR'] == 'Compromisso'), 'CALENDAR'] = 'Saúde' # 2
print(f'Valores de CALENDAR para "Psicóloga": {df.loc[df["SUMMARY"] == "Psicóloga", "CALENDAR"].unique()}')

# Corrigindo valores duplicados de 'SO: P1' # 2
df.loc[(df['SUMMARY'] == 'SO: P1') & (df['CALENDAR'] == 'PUC'), 'CALENDAR'] = 'Exames' # 2
print(f'Valores de CALENDAR para "SO: P1": {df.loc[df["SUMMARY"] == "SO: P1", "CALENDAR"].unique()}')

=== TRATANDO Duplicatas da coluna SUMMARY ===
Valores de CALENDAR para "Academia": ['Exercícios']
Valores de CALENDAR para "Aldeia": ['Social']
Valores de CALENDAR para "Aula": ['PUC']
Valores de CALENDAR para "BD: T2 G2": ['Exames']
Valores de CALENDAR para "BIRUSAMBA DE VERÃO": ['Social']
Valores de CALENDAR para "De-Para": ['PUC']
Valores de CALENDAR para "Muay Thai": ['Exercícios']
Valores de CALENDAR para "Psicóloga": ['Saúde']
Valores de CALENDAR para "SO: P1": ['Exames']


In [8]:
print("=== ENCONTRO DE DUPLICATAS DA COLUNA SUMMARY ===")
duplicates = df[df.duplicated(subset='SUMMARY', keep=False)]
# Seleciona duplicatas de SUMMARY que aparecem em mais de um CALENDAR
duplicates_sum_cal = (
    duplicates[['SUMMARY', 'CALENDAR']]
    .drop_duplicates()
    .groupby('SUMMARY')
    .filter(lambda x: x['CALENDAR'].nunique() > 1)
    .sort_values(by=['SUMMARY', 'CALENDAR'], ascending=[True, True])
)
print(f"Quantidade de duplicatas encontradas: {duplicates_sum_cal.shape[0]}")
print(f"Duplicatas encontradas:\n{duplicates_sum_cal}")

=== ENCONTRO DE DUPLICATAS DA COLUNA SUMMARY ===
Quantidade de duplicatas encontradas: 4
Duplicatas encontradas:
         SUMMARY     CALENDAR
2862  Alessandra  Compromisso
3059  Alessandra        Saúde
4566    Itaipava      Pessoal
5951    Itaipava       Social


### Entendendo os valores unicos da coluna CALENDAR

In [9]:
print('=== ENTENDENDO OS VALORES ÚNICOS DA COLUNA CALENDAR ===')
print(f"Valores únicos na coluna CALENDAR: {df['CALENDAR'].unique()}")

# Amostra de linhas com valor 'Rotina'
sample_rotina = df[df['CALENDAR'] == 'Rotina']['SUMMARY'].unique()
print(f"\nAmostra de linhas com CALENDAR 'Rotina':\n{sample_rotina}")

=== ENTENDENDO OS VALORES ÚNICOS DA COLUNA CALENDAR ===
Valores únicos na coluna CALENDAR: ['Alimentação' 'PUC' 'Pessoal' 'Compromisso' 'Saúde' 'Exames' 'Social'
 'Games' 'Exercícios' 'RescueTime' 'Rotina' 'Trabalho' 'Viagens']

Amostra de linhas com CALENDAR 'Rotina':
['Deep Work']


# Exportar DataFrame completo para CSV

In [10]:
# Salvar dataset processado
if df is not None:
    print("=== SALVANDO DATASET PROCESSADO ===")

    # Criar algumas colunas derivadas úteis
    df['YEAR'] = df['DTSTART'].dt.year
    df['MONTH'] = df['DTSTART'].dt.month
    df['DAY_OF_WEEK'] = df['DTSTART'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['HOUR'] = df['DTSTART'].dt.hour
    df['DATE'] = df['DTSTART'].dt.date

    # Criar coluna de duração em minutos (se não existe DURATION)
    if 'DTSTART' in df.columns and 'DTEND' in df.columns:
        df['DURATION_MINUTES'] = (df['DTEND'] - df['DTSTART']).dt.total_seconds() / 60

    print(f"Colunas adicionadas: YEAR, MONTH, DAY_OF_WEEK, HOUR, DATE, DURATION_MINUTES")

    # Salvar como CSV processado
    output_path = '../data/tasks_processed.csv'
    df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"✅ Dataset processado salvo em: {output_path}")

    # Estatísticas finais
    print(f"\n=== ESTATÍSTICAS FINAIS ===")
    print(f"Total de registros: {len(df)}")
    print(f"Período dos dados: {df['DTSTART'].min()} até {df['DTSTART'].max()}")
    print(f"Categorias únicas: {df['CALENDAR'].nunique()}")
    print(f"Duração média das tarefas: {df['DURATION_MINUTES'].mean():.1f} minutos")
else:
    print("❌ DataFrame não disponível para salvar.")

=== SALVANDO DATASET PROCESSADO ===
Colunas adicionadas: YEAR, MONTH, DAY_OF_WEEK, HOUR, DATE, DURATION_MINUTES
✅ Dataset processado salvo em: ../data/tasks_processed.csv

=== ESTATÍSTICAS FINAIS ===
Total de registros: 7232
Período dos dados: 2002-04-22 00:00:00 até 2026-12-30 15:00:00
Categorias únicas: 13
Duração média das tarefas: 192.2 minutos
